In [1]:
%reload_ext autoreload
%autoreload 2


import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from pathlib import Path
from MHW_metrics import main

import sys
sys.path.append("..")
import utils


In [2]:
# Available models once they've been downloaded

models = sorted(list(utils.dirtodict('../../data/CMIP6/').keys()))[1:]

In [ ]:
def open_and_process_dataset(files, time_slice=None):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        ds = xr.open_mfdataset(files, combine="by_coords")
        if time_slice:
            ds = ds.sel(time=ds.time.dt.year.isin(list(range(*time_slice))))
    return ds

# Set the baseline type and baseline length

baseline_year_length = 20
baseline_type = "moving_baseline"

for model in models:
    print('\n')
    print(model)
    data_path = Path(f"../../data/CMIP6/{model}/")
    dir_dict = utils.dirtodict(data_path)
    variants = sorted(list(dir_dict.keys())[:-1])

    for variant in variants:
        print(variant)
        files_hist = dir_dict[variant]["tos"]["historical"][".files"]
        ds_hist = open_and_process_dataset(files_hist, time_slice=(1982, 2015))

        for experiment in ["ssp126", "ssp585"]:
            print(experiment)

            output_dir = Path(f"../../results/MHW/CMIP6/{model}/{variant}/{baseline_type}/")
            output_dir.mkdir(parents=True, exist_ok=True)
            output_path = output_dir / f"MHW_metrics_{experiment}.nc"

            if os.path.exists(output_path):
                print(f"Skipping")
                continue

            files_ssp = dir_dict[variant]["tos"][experiment][".files"]
            ds_ssp = open_and_process_dataset(files_ssp, time_slice=(2015, 2100))

            ds = xr.concat([ds_hist, ds_ssp], dim="time")

            ds_mhw = main.MHW_metrics_cmip(ds, baseline_year_length, baseline_type=baseline_type)

            ds_mhw.to_netcdf(output_path)